In [1]:
import os
import requests
import dotenv
import time
import numpy as np
import pandas as pd
import yfinance
import alpaca_trade_api as alpaca_api
import krakenex
import pykrakenapi
import pandas_montecarlo

%matplotlib inline

In [2]:
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame({'ticker':tickers})
    for n in range(0, n):
        random_portfolios[f'random weights {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'random weights {n+1}'] = random_portfolios[f'random weights {n+1}'] / sum(random_portfolios[f'random weights {n+1}'])
    return(random_portfolios)

In [3]:
dotenv.load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv('alpaca_api_key')
alpaca_secret_key = os.getenv('alpaca_secret_key')
kraken_api_key = os.getenv('kraken_api_key')
kraken_secret_key = os.getenv('kraken_secret_key')

print(type(alpaca_api_key))
print(type(alpaca_secret_key))
print(type(kraken_api_key))
print(type(kraken_secret_key))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [5]:
#table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#sp_500_table = table[0]
#sp_500_list = sp_500_table['Symbol'].to_list()
#sp_500 = yfinance.download(sp_500_list, '2015-12-01')
#sp_500

In [6]:
alpaca = alpaca_api.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

faangmula_tickers =  ['AAPL',  'ABNB', 'AMZN', 'FB', 'GOOG', 'LYFT', 'MSFT', 'NFLX', 'UBER']
start = pd.Timestamp('2015-12-01', tz='America/New_York').isoformat()

faangmula_original = alpaca.get_bars(faangmula_tickers, alpaca_api.TimeFrame.Day, start = start).df
faangmula_original

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,118.73,118.810,116.860,117.34,34852374,187129,117.756760,AAPL
2015-12-02 05:00:00+00:00,117.00,118.110,116.080,116.28,33385643,180616,117.151198,AAPL
2015-12-03 05:00:00+00:00,116.55,116.790,114.220,115.20,41560785,245330,115.434888,AAPL
2015-12-04 05:00:00+00:00,115.20,119.250,115.110,119.03,57776977,307788,118.187290,AAPL
2015-12-07 05:00:00+00:00,118.98,119.860,117.810,118.28,32080754,190809,118.509111,AAPL
...,...,...,...,...,...,...,...,...
2022-04-04 04:00:00+00:00,36.40,36.930,36.215,36.51,20197208,146275,36.621379,UBER
2022-04-05 04:00:00+00:00,36.43,36.610,34.720,34.96,20987999,153337,35.188058,UBER
2022-04-06 04:00:00+00:00,33.82,34.000,32.550,33.41,29163174,203021,33.287397,UBER


In [7]:
faangmula = faangmula_original.drop(['open', 'high', 'low', 'volume', 'trade_count', 'vwap'], axis = 1).pivot(columns = 'symbol')
faangmula.index = faangmula.index.date
faangmula.columns = faangmula_tickers
faangmula

,AAPL,ABNB,AMZN,FB,GOOG,LYFT,MSFT,NFLX,UBER
2015-12-01,117.34,NaN,679.06,107.12,767.04,NaN,55.22,125.37,NaN
2015-12-02,116.28,NaN,676.01,106.07,762.38,NaN,55.21,128.93,NaN
2015-12-03,115.20,NaN,666.25,104.38,752.54,NaN,54.20,126.81,NaN
2015-12-04,119.03,NaN,672.64,106.18,766.81,NaN,55.91,130.93,NaN
2015-12-07,118.28,NaN,669.83,105.61,763.25,NaN,55.81,125.36,NaN
...,...,...,...,...,...,...,...,...,...
2022-04-04,178.44,177.02,3366.93,233.89,2872.85,38.96,314.97,391.50,36.51
2022-04-05,175.06,171.21,3281.10,231.84,2821.26,37.15,310.88,380.15,34.96
2022-04-06,171.59,164.66,3175.12,223.30,2743.52,36.08,299.50,368.35,33.41
2022-04-07,172.14,165.91,3155.69,222.95,2729.30,34.61,301.37,362.15,32.27


In [8]:
# Market capitalization data in trillions of dollars, sourced from Trading View on April 6, 2022.

faangmula_caps_list = [2.804, 0.105855, 1.616, 0.60781, 1.81, 0.012576, 2.245, 0.163534, 0.065299]
faangmula_cap_portfolio = pd.DataFrame({'tickers':faangmula_tickers, 'market cap weights':[market_cap / sum(faangmula_caps_list) for market_cap in faangmula_caps_list]}).set_index('tickers')
faangmula_cap_portfolio

,market cap weights
tickers,
AAPL,0.297347
ABNB,0.011225
AMZN,0.171367
FB,0.064454
GOOG,0.191939
LYFT,0.001334
MSFT,0.238068
NFLX,0.017342
UBER,0.006925


In [9]:
faangmula_cap_daily_returns = faangmula.pct_change().fillna(0.0).dot(faangmula_cap_portfolio)
faangmula_cap_daily_returns.columns = ['faangmula index']
faangmula_cap_daily_returns

,faangmula index
2015-12-01,0.000000
2015-12-02,-0.004804
2015-12-03,-0.013380
2015-12-04,0.024355
2015-12-07,-0.004990
...,...
2022-04-04,0.024182
2022-04-05,-0.018331
2022-04-06,-0.029120
2022-04-07,-0.000202


In [10]:
random_faangmula_portfolios = random_portfolios_generator(faangmula_tickers, 1000)
random_faangmula_portfolios

/usr/local/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


,ticker,random weights 1,random weights 2,random weights 3,random weights 4,random weights 5,random weights 6,random weights 7,random weights 8,random weights 9,...,random weights 991,random weights 992,random weights 993,random weights 994,random weights 995,random weights 996,random weights 997,random weights 998,random weights 999,random weights 1000
0,AAPL,0.032217,0.209609,0.189647,0.078216,0.015573,0.219732,0.230162,0.057191,0.182103,...,0.014526,0.170535,0.094967,0.104676,0.071361,0.048973,0.115524,0.140376,0.032990,0.039640
1,ABNB,0.267891,0.058976,0.025833,0.077189,0.036485,0.017829,0.104160,0.030612,0.124060,...,0.132371,0.066255,0.064892,0.066708,0.109183,0.132605,0.053401,0.155515,0.062020,0.122500
2,AMZN,0.048220,0.043124,0.054814,0.136352,0.080506,0.210573,0.105318,0.128834,0.161772,...,0.259793,0.041693,0.180941,0.052980,0.084846,0.180921,0.077678,0.072015,0.208789,0.036783
3,FB,0.107046,0.161089,0.211473,0.074116,0.186313,0.063373,0.183726,0.194985,0.058553,...,0.128889,0.097198,0.189184,0.051039,0.044216,0.080346,0.168275,0.157040,0.097059,0.070558
4,GOOG,0.170932,0.050889,0.036999,0.114527,0.188347,0.103462,0.080717,0.236203,0.116819,...,0.114806,0.076066,0.115741,0.191490,0.075992,0.057311,0.040205,0.137977,0.040008,0.217404
5,LYFT,0.041247,0.114874,0.011929,0.108514,0.160300,0.027577,0.118153,0.052794,0.082309,...,0.059978,0.012259,0.090854,0.136468,0.210470,0.098275,0.018640,0.045375,0.025776,0.192213
6,MSFT,0.039529,0.011767,0.044165,0.080874,0.126860,0.132648,0.073511,0.197231,0.081080,...,0.041637,0.078512,0.013279,0.132429,0.127576,0.207057,0.152501,0.184223,0.210668,0.206341
7,NFLX,0.019891,0.149401,0.199916,0.147680,0.110154,0.174674,0.088218,0.098058,0.080181,...,0.138511,0.236258,0.153409,0.106274,0.051255,0.105675,0.214911,0.074596,0.130368,0.007982
8,UBER,0.273027,0.200271,0.225226,0.182531,0.095462,0.050132,0.016036,0.004093,0.113123,...,0.109487,0.221225,0.096732,0.157935,0.225102,0.088836,0.158864,0.032883,0.192321,0.106581


In [11]:
random_faangmula_daily_returns_list = []
for n in range(0, (len(random_faangmula_portfolios.columns) - 1)):
        random_faangmula_daily_returns_list.append(faangmula.pct_change().fillna(0.0).dot(list(random_faangmula_portfolios.loc[:, f'random weights {n + 1}'])))

random_faangmula_daily_returns = pd.concat(random_faangmula_daily_returns_list, axis = 'columns', join = 'inner')

for n in range(0, (len(random_faangmula_portfolios.columns) - 1)):
        random_faangmula_daily_returns = random_faangmula_daily_returns.rename(columns = {n: f'random faangmula portfolio {n + 1}'})

random_faangmula_daily_returns

,random faangmula portfolio 1,random faangmula portfolio 2,random faangmula portfolio 3,random faangmula portfolio 4,random faangmula portfolio 5,random faangmula portfolio 6,random faangmula portfolio 7,random faangmula portfolio 8,random faangmula portfolio 9,random faangmula portfolio 10,...,random faangmula portfolio 991,random faangmula portfolio 992,random faangmula portfolio 993,random faangmula portfolio 994,random faangmula portfolio 995,random faangmula portfolio 996,random faangmula portfolio 997,random faangmula portfolio 998,random faangmula portfolio 999,random faangmula portfolio 1000
2015-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-02,-0.002038,0.000265,0.001412,0.001438,-0.000368,0.000755,-0.002352,-0.001693,-0.001393,0.000999,...,0.000667,0.003552,0.000126,0.000147,-0.000488,0.000572,0.002789,-0.001884,0.001234,-0.002346
2015-12-03,-0.005957,-0.008465,-0.010495,-0.009262,-0.010838,-0.012725,-0.010423,-0.013767,-0.009269,-0.008232,...,-0.010460,-0.010037,-0.010768,-0.009192,-0.006750,-0.010612,-0.011718,-0.011223,-0.011381,-0.008735
2015-12-04,0.008514,0.016350,0.019068,0.014708,0.015656,0.022240,0.018546,0.020387,0.015994,0.016690,...,0.013188,0.019341,0.015753,0.016130,0.011080,0.015802,0.020044,0.018918,0.016414,0.013779
2015-12-07,-0.002690,-0.008979,-0.011315,-0.008419,-0.007222,-0.010753,-0.007136,-0.007566,-0.006236,-0.009839,...,-0.008369,-0.012315,-0.009457,-0.006802,-0.003803,-0.006628,-0.011558,-0.006172,-0.007710,-0.002500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,0.023441,0.029898,0.030182,0.028414,0.030139,0.029096,0.030336,0.028961,0.026828,0.032756,...,0.029899,0.028815,0.031609,0.026680,0.026189,0.027806,0.030002,0.027314,0.026941,0.025228
2022-04-05,-0.029237,-0.027654,-0.024841,-0.028319,-0.024901,-0.022725,-0.023205,-0.019353,-0.026260,-0.028008,...,-0.026425,-0.026516,-0.025225,-0.027255,-0.031327,-0.025715,-0.023939,-0.021512,-0.025430,-0.026597
2022-04-06,-0.036455,-0.032396,-0.033479,-0.033556,-0.033270,-0.030319,-0.030484,-0.032123,-0.031800,-0.030794,...,-0.034060,-0.033307,-0.032521,-0.032574,-0.034358,-0.034055,-0.033982,-0.032469,-0.035476,-0.033313
2022-04-07,-0.010296,-0.013616,-0.011315,-0.013338,-0.012284,-0.006030,-0.006209,-0.004612,-0.008218,-0.013427,...,-0.009586,-0.011287,-0.010708,-0.012466,-0.016247,-0.007888,-0.009017,-0.002848,-0.009552,-0.010720


In [12]:
faangmula_daily_returns = pd.concat([faangmula_cap_daily_returns, random_faangmula_daily_returns], axis = 1, join = 'inner')
faangmula_daily_returns

,faangmula index,random faangmula portfolio 1,random faangmula portfolio 2,random faangmula portfolio 3,random faangmula portfolio 4,random faangmula portfolio 5,random faangmula portfolio 6,random faangmula portfolio 7,random faangmula portfolio 8,random faangmula portfolio 9,...,random faangmula portfolio 991,random faangmula portfolio 992,random faangmula portfolio 993,random faangmula portfolio 994,random faangmula portfolio 995,random faangmula portfolio 996,random faangmula portfolio 997,random faangmula portfolio 998,random faangmula portfolio 999,random faangmula portfolio 1000
2015-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-02,-0.004804,-0.002038,0.000265,0.001412,0.001438,-0.000368,0.000755,-0.002352,-0.001693,-0.001393,...,0.000667,0.003552,0.000126,0.000147,-0.000488,0.000572,0.002789,-0.001884,0.001234,-0.002346
2015-12-03,-0.013380,-0.005957,-0.008465,-0.010495,-0.009262,-0.010838,-0.012725,-0.010423,-0.013767,-0.009269,...,-0.010460,-0.010037,-0.010768,-0.009192,-0.006750,-0.010612,-0.011718,-0.011223,-0.011381,-0.008735
2015-12-04,0.024355,0.008514,0.016350,0.019068,0.014708,0.015656,0.022240,0.018546,0.020387,0.015994,...,0.013188,0.019341,0.015753,0.016130,0.011080,0.015802,0.020044,0.018918,0.016414,0.013779
2015-12-07,-0.004990,-0.002690,-0.008979,-0.011315,-0.008419,-0.007222,-0.010753,-0.007136,-0.007566,-0.006236,...,-0.008369,-0.012315,-0.009457,-0.006802,-0.003803,-0.006628,-0.011558,-0.006172,-0.007710,-0.002500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,0.024182,0.023441,0.029898,0.030182,0.028414,0.030139,0.029096,0.030336,0.028961,0.026828,...,0.029899,0.028815,0.031609,0.026680,0.026189,0.027806,0.030002,0.027314,0.026941,0.025228
2022-04-05,-0.018331,-0.029237,-0.027654,-0.024841,-0.028319,-0.024901,-0.022725,-0.023205,-0.019353,-0.026260,...,-0.026425,-0.026516,-0.025225,-0.027255,-0.031327,-0.025715,-0.023939,-0.021512,-0.025430,-0.026597
2022-04-06,-0.029120,-0.036455,-0.032396,-0.033479,-0.033556,-0.033270,-0.030319,-0.030484,-0.032123,-0.031800,...,-0.034060,-0.033307,-0.032521,-0.032574,-0.034358,-0.034055,-0.033982,-0.032469,-0.035476,-0.033313
2022-04-07,-0.000202,-0.010296,-0.013616,-0.011315,-0.013338,-0.012284,-0.006030,-0.006209,-0.004612,-0.008218,...,-0.009586,-0.011287,-0.010708,-0.012466,-0.016247,-0.007888,-0.009017,-0.002848,-0.009552,-0.010720


In [13]:
faangmula_correlations = faangmula_daily_returns.corr()
faangmula_correlations

,faangmula index,random faangmula portfolio 1,random faangmula portfolio 2,random faangmula portfolio 3,random faangmula portfolio 4,random faangmula portfolio 5,random faangmula portfolio 6,random faangmula portfolio 7,random faangmula portfolio 8,random faangmula portfolio 9,...,random faangmula portfolio 991,random faangmula portfolio 992,random faangmula portfolio 993,random faangmula portfolio 994,random faangmula portfolio 995,random faangmula portfolio 996,random faangmula portfolio 997,random faangmula portfolio 998,random faangmula portfolio 999,random faangmula portfolio 1000
faangmula index,1.000000,0.695427,0.848330,0.880365,0.829472,0.827065,0.963942,0.944223,0.926862,0.919847,...,0.837119,0.870187,0.880447,0.846573,0.709066,0.871607,0.888141,0.945892,0.858195,0.791139
random faangmula portfolio 1,0.695427,1.000000,0.913469,0.872606,0.934486,0.913082,0.758620,0.844081,0.796017,0.901863,...,0.893138,0.877838,0.882821,0.933981,0.956743,0.910037,0.860186,0.856890,0.896639,0.947956
random faangmula portfolio 2,0.848330,0.913469,1.000000,0.980819,0.976895,0.950135,0.910372,0.958553,0.882069,0.973717,...,0.920476,0.976368,0.960441,0.977981,0.941745,0.944503,0.957491,0.928936,0.938882,0.932351
random faangmula portfolio 3,0.880365,0.872606,0.980819,1.000000,0.966197,0.944636,0.949027,0.959846,0.924453,0.965133,...,0.941762,0.992521,0.976872,0.953626,0.878640,0.948182,0.989234,0.948155,0.960008,0.885733
random faangmula portfolio 4,0.829472,0.934486,0.976895,0.966197,1.000000,0.980134,0.912338,0.935620,0.912551,0.972688,...,0.968775,0.973608,0.977639,0.990704,0.955148,0.981940,0.968077,0.932299,0.980351,0.956847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
random faangmula portfolio 996,0.871607,0.910037,0.944503,0.948182,0.981940,0.975064,0.936362,0.949486,0.951280,0.975740,...,0.984775,0.954634,0.981120,0.970713,0.914777,1.000000,0.969997,0.965569,0.988741,0.947314
random faangmula portfolio 997,0.888141,0.860186,0.957491,0.989234,0.968077,0.955475,0.963822,0.954804,0.955389,0.962164,...,0.966259,0.989213,0.985605,0.950486,0.861317,0.969997,1.000000,0.962716,0.979758,0.886917
random faangmula portfolio 998,0.945892,0.856890,0.928936,0.948155,0.932299,0.938856,0.962857,0.983236,0.973882,0.973411,...,0.944895,0.941925,0.965625,0.935569,0.837150,0.965569,0.962716,1.000000,0.945939,0.904486
random faangmula portfolio 999,0.858195,0.896639,0.938882,0.960008,0.980351,0.968222,0.935580,0.925729,0.947651,0.959230,...,0.985051,0.964051,0.979706,0.959590,0.897581,0.988741,0.979758,0.945939,1.000000,0.919689


In [26]:
low_correlation_faangmula_original = faangmula_correlations.sort_values('faangmula index')['faangmula index'].head(10).reset_index().rename({'index': 'portfolio', 'faangmula index': 'faangmula index correlation'}, axis = 1)
low_correlation_faangmula_original.loc[len(low_correlation_faangmula_original.values)] = ['faangmula index', 1.0]
low_correlation_faangmula_original

,portfolio,faangmula index correlation
0,random faangmula portfolio 216,0.625890
1,random faangmula portfolio 928,0.656087
2,random faangmula portfolio 232,0.659532
3,random faangmula portfolio 676,0.660092
4,random faangmula portfolio 208,0.677279
5,random faangmula portfolio 885,0.683748
6,random faangmula portfolio 347,0.684206
7,random faangmula portfolio 456,0.689131
8,random faangmula portfolio 66,0.691361
9,random faangmula portfolio 802,0.693490


In [27]:
variances = []
covariances = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    variance = faangmula_daily_returns[f'{portfolio}'].var()
    covariance = faangmula_daily_returns[f'{portfolio}'].cov(faangmula_daily_returns['faangmula index'])
    variances.append(variance)
    covariances.append(covariance)

lc_intermediate_1 = pd.concat([low_correlation_faangmula_original, pd.DataFrame(variances)], axis = 1).rename({0: 'portfolio variance'}, axis = 1)
lc_intermediate_2 = pd.concat([lc_intermediate_1, pd.DataFrame(covariances)], axis = 1).rename({0: 'faangmula index covariance'}, axis = 1)
lc_intermediate_2

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance
0,random faangmula portfolio 216,0.625890,0.000256,0.000163
1,random faangmula portfolio 928,0.656087,0.000229,0.000162
2,random faangmula portfolio 232,0.659532,0.000238,0.000166
3,random faangmula portfolio 676,0.660092,0.000311,0.000190
4,random faangmula portfolio 208,0.677279,0.000226,0.000166
5,random faangmula portfolio 885,0.683748,0.000251,0.000176
6,random faangmula portfolio 347,0.684206,0.000292,0.000191
7,random faangmula portfolio 456,0.689131,0.000202,0.000160
8,random faangmula portfolio 66,0.691361,0.000205,0.000161
9,random faangmula portfolio 802,0.693490,0.000182,0.000152


In [28]:
betas = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    beta = lc_intermediate_2.set_index('portfolio').at[f'{portfolio}', 'faangmula index covariance'] / lc_intermediate_2.set_index('portfolio').at[f'{portfolio}', 'portfolio variance']
    betas.append(beta)

lc_intermediate_3 = pd.concat([lc_intermediate_2, pd.DataFrame(betas)], axis = 1).rename({0: 'faangmula index beta'}, axis = 1)
lc_intermediate_3

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance,faangmula index beta
0,random faangmula portfolio 216,0.625890,0.000256,0.000163,0.637896
1,random faangmula portfolio 928,0.656087,0.000229,0.000162,0.706981
2,random faangmula portfolio 232,0.659532,0.000238,0.000166,0.696729
3,random faangmula portfolio 676,0.660092,0.000311,0.000190,0.610018
4,random faangmula portfolio 208,0.677279,0.000226,0.000166,0.734621
5,random faangmula portfolio 885,0.683748,0.000251,0.000176,0.703964
6,random faangmula portfolio 347,0.684206,0.000292,0.000191,0.652122
7,random faangmula portfolio 456,0.689131,0.000202,0.000160,0.790242
8,random faangmula portfolio 66,0.691361,0.000205,0.000161,0.786637
9,random faangmula portfolio 802,0.693490,0.000182,0.000152,0.837868


In [29]:
average_returns = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    average_return = faangmula_daily_returns[f'{portfolio}'].mean()
    average_returns.append(average_return)

lc_intermediate_4 = pd.concat([lc_intermediate_3, pd.DataFrame(average_returns)], axis = 1).rename({0: 'average daily return'}, axis = 1)
lc_intermediate_4

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance,faangmula index beta,average daily return
0,random faangmula portfolio 216,0.625890,0.000256,0.000163,0.637896,0.000358
1,random faangmula portfolio 928,0.656087,0.000229,0.000162,0.706981,0.000483
2,random faangmula portfolio 232,0.659532,0.000238,0.000166,0.696729,0.000437
3,random faangmula portfolio 676,0.660092,0.000311,0.000190,0.610018,0.000406
4,random faangmula portfolio 208,0.677279,0.000226,0.000166,0.734621,0.000471
5,random faangmula portfolio 885,0.683748,0.000251,0.000176,0.703964,0.000505
6,random faangmula portfolio 347,0.684206,0.000292,0.000191,0.652122,0.000467
7,random faangmula portfolio 456,0.689131,0.000202,0.000160,0.790242,0.000502
8,random faangmula portfolio 66,0.691361,0.000205,0.000161,0.786637,0.000435
9,random faangmula portfolio 802,0.693490,0.000182,0.000152,0.837868,0.000512


In [31]:
stds = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    std = faangmula_daily_returns[f'{portfolio}'].std()
    stds.append(std)

lc_intermediate_5 = pd.concat([lc_intermediate_4, pd.DataFrame(stds)], axis = 1).rename({0: 'standard deviation'}, axis = 1)
lc_intermediate_5

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance,faangmula index beta,average daily return,standard deviation
0,random faangmula portfolio 216,0.625890,0.000256,0.000163,0.637896,0.000358,0.015991
1,random faangmula portfolio 928,0.656087,0.000229,0.000162,0.706981,0.000483,0.015124
2,random faangmula portfolio 232,0.659532,0.000238,0.000166,0.696729,0.000437,0.015428
3,random faangmula portfolio 676,0.660092,0.000311,0.000190,0.610018,0.000406,0.017635
4,random faangmula portfolio 208,0.677279,0.000226,0.000166,0.734621,0.000471,0.015026
5,random faangmula portfolio 885,0.683748,0.000251,0.000176,0.703964,0.000505,0.015830
6,random faangmula portfolio 347,0.684206,0.000292,0.000191,0.652122,0.000467,0.017099
7,random faangmula portfolio 456,0.689131,0.000202,0.000160,0.790242,0.000502,0.014212
8,random faangmula portfolio 66,0.691361,0.000205,0.000161,0.786637,0.000435,0.014324
9,random faangmula portfolio 802,0.693490,0.000182,0.000152,0.837868,0.000512,0.013489


In [32]:
zero_sharpes = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    zero_sharpe = (lc_intermediate_5.set_index('portfolio').at[f'{portfolio}', 'average daily return'] - 0) / lc_intermediate_5.set_index('portfolio').at[f'{portfolio}', 'standard deviation']
    zero_sharpes.append(zero_sharpe)

lc_intermediate_6 = pd.concat([lc_intermediate_5, pd.DataFrame(zero_sharpes)], axis = 1).rename({0: 'zero sharpe ratio'}, axis = 1)
lc_intermediate_6

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance,faangmula index beta,average daily return,standard deviation,zero sharpe ratio
0,random faangmula portfolio 216,0.625890,0.000256,0.000163,0.637896,0.000358,0.015991,0.022368
1,random faangmula portfolio 928,0.656087,0.000229,0.000162,0.706981,0.000483,0.015124,0.031909
2,random faangmula portfolio 232,0.659532,0.000238,0.000166,0.696729,0.000437,0.015428,0.028314
3,random faangmula portfolio 676,0.660092,0.000311,0.000190,0.610018,0.000406,0.017635,0.023030
4,random faangmula portfolio 208,0.677279,0.000226,0.000166,0.734621,0.000471,0.015026,0.031341
5,random faangmula portfolio 885,0.683748,0.000251,0.000176,0.703964,0.000505,0.015830,0.031928
6,random faangmula portfolio 347,0.684206,0.000292,0.000191,0.652122,0.000467,0.017099,0.027286
7,random faangmula portfolio 456,0.689131,0.000202,0.000160,0.790242,0.000502,0.014212,0.035297
8,random faangmula portfolio 66,0.691361,0.000205,0.000161,0.786637,0.000435,0.014324,0.030388
9,random faangmula portfolio 802,0.693490,0.000182,0.000152,0.837868,0.000512,0.013489,0.037966


In [46]:
faangmula_sharpes = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    faangmula_sharpe = (lc_intermediate_6.set_index('portfolio').at[f'{portfolio}', 'average daily return'] - lc_intermediate_5.set_index('portfolio').at['faangmula index', 'average daily return']) / lc_intermediate_6.set_index('portfolio').at[f'{portfolio}', 'standard deviation']
    faangmula_sharpes.append(faangmula_sharpe)
lc_intermediate_7 = pd.concat([lc_intermediate_6, pd.DataFrame(faangmula_sharpes)], axis = 1).rename({0: 'faangmula sharpe ratio'}, axis = 1)
lc_intermediate_7

#barplot for index correlations, scatter cumprod vs std, barplot for zero sharpe ratio, bar beta, maybe scatter std vs beta, maybe efficient frontier

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance,faangmula index beta,average daily return,standard deviation,zero sharpe ratio,faangmula sharpe ratio
0,random faangmula portfolio 216,0.625890,0.000256,0.000163,0.637896,0.000358,0.015991,0.022368,-0.037233
1,random faangmula portfolio 928,0.656087,0.000229,0.000162,0.706981,0.000483,0.015124,0.031909,-0.031107
2,random faangmula portfolio 232,0.659532,0.000238,0.000166,0.696729,0.000437,0.015428,0.028314,-0.033464
3,random faangmula portfolio 676,0.660092,0.000311,0.000190,0.610018,0.000406,0.017635,0.023030,-0.031013
4,random faangmula portfolio 208,0.677279,0.000226,0.000166,0.734621,0.000471,0.015026,0.031341,-0.032090
5,random faangmula portfolio 885,0.683748,0.000251,0.000176,0.703964,0.000505,0.015830,0.031928,-0.028280
6,random faangmula portfolio 347,0.684206,0.000292,0.000191,0.652122,0.000467,0.017099,0.027286,-0.028451
7,random faangmula portfolio 456,0.689131,0.000202,0.000160,0.790242,0.000502,0.014212,0.035297,-0.031763
8,random faangmula portfolio 66,0.691361,0.000205,0.000161,0.786637,0.000435,0.014324,0.030388,-0.036151
9,random faangmula portfolio 802,0.693490,0.000182,0.000152,0.837868,0.000512,0.013489,0.037966,-0.032689


In [35]:
from datetime import datetime
#import questionary
import requests
import json

In [52]:
def coinapi_data(tickers):
   
    headers = {'X-CoinAPI-Key' : '49990642-7927-4390-8BB1-A2D2D096B94C'}

    coinapi_data = {}
    for ticker in tickers:
        url = f'https://rest.coinapi.io/v1/exchangerate/{ticker}/USD/history?period_id=1DAY&time_start=2022-01-01T00:00:00&time_end=2022-02-01T00:00:00&limit=900'
        response = requests.get(url, headers=headers)
        
        ticker_data = pd.DataFrame(json.loads(response.content))

        #ticker_data['symbol'] = ticker
        ticker_data = ticker_data.rename(columns = {'time_period_end': 'timestamp', 'rate_close': 'close'})
        ticker_data = ticker_data.drop(['time_period_start', 'time_open', 'time_close','rate_open', 'rate_high', 'rate_low'], axis=1).reset_index()
        ticker_data = ticker_data.set_index('timestamp')
        coinapi_data[ticker] = ticker_data

        time.sleep(1)
    return coinapi_data

In [65]:
tickers = ['BTC','ETH','LUNA','SOL','XRP','ADA','AVAX','DOT','DOGE','SHIB','MATIC', 'DAI','LTC','ATOM','LINK','UNI','TRX','BCH']
two_tickers = ['BTC','ETH']
crypto_original = coinapi_data(two_tickers)
crypto_original

{'BTC':                               index         close
 timestamp                                        
 2022-01-02T00:00:00.0000000Z      0  47756.298649
 2022-01-03T00:00:00.0000000Z      1  47335.070977
 2022-01-04T00:00:00.0000000Z      2  46434.298793
 2022-01-05T00:00:00.0000000Z      3  45850.680183
 2022-01-06T00:00:00.0000000Z      4  43499.079007
 2022-01-07T00:00:00.0000000Z      5  43147.092367
 2022-01-08T00:00:00.0000000Z      6  41522.177853
 2022-01-09T00:00:00.0000000Z      7  41707.240948
 2022-01-10T00:00:00.0000000Z      8  41873.145510
 2022-01-11T00:00:00.0000000Z      9  41843.275617
 2022-01-12T00:00:00.0000000Z     10  42708.770107
 2022-01-13T00:00:00.0000000Z     11  43907.509543
 2022-01-14T00:00:00.0000000Z     12  42521.414580
 2022-01-15T00:00:00.0000000Z     13  43087.278288
 2022-01-16T00:00:00.0000000Z     14  43122.543047
 2022-01-17T00:00:00.0000000Z     15  43097.790711
 2022-01-18T00:00:00.0000000Z     16  42247.248880
 2022-01-19T00:00:00.000

In [ ]:
crypto_original = pd.concat(crypto_original.values(), keys = crypto_original.keys(), axis = 'columns')
crypto_original = crypto_original.xs('close', level = 1, axis = 'columns')
crypto_original

In [76]:
crypto_original.index = crypto_original.index.astype('datetime64[ns]')

In [77]:
all_close = pd.concat([faangmula, crypto_original], axis = 1, join = 'outer')
all_close

/usr/local/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexes/base.py:345: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  joined_ndarray, lidx, ridx = libjoin.outer_join_indexer(sv, ov)
/usr/local/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexes/base.py:3512: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  indexer = self._engine.get_indexer(target._get_engine_target())


,AAPL,ABNB,AMZN,FB,GOOG,LYFT,MSFT,NFLX,UBER,BTC,ETH,BTC,ETH
2015-12-01,117.34,NaN,679.06,107.12,767.04,NaN,55.22,125.37,NaN,NaN,NaN,NaN,NaN
2015-12-02,116.28,NaN,676.01,106.07,762.38,NaN,55.21,128.93,NaN,NaN,NaN,NaN,NaN
2015-12-03,115.20,NaN,666.25,104.38,752.54,NaN,54.20,126.81,NaN,NaN,NaN,NaN,NaN
2015-12-04,119.03,NaN,672.64,106.18,766.81,NaN,55.91,130.93,NaN,NaN,NaN,NaN,NaN
2015-12-07,118.28,NaN,669.83,105.61,763.25,NaN,55.81,125.36,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,178.44,177.02,3366.93,233.89,2872.85,38.96,314.97,391.50,36.51,NaN,NaN,NaN,NaN
2022-04-05,175.06,171.21,3281.10,231.84,2821.26,37.15,310.88,380.15,34.96,NaN,NaN,NaN,NaN
2022-04-06,171.59,164.66,3175.12,223.30,2743.52,36.08,299.50,368.35,33.41,NaN,NaN,NaN,NaN
2022-04-07,172.14,165.91,3155.69,222.95,2729.30,34.61,301.37,362.15,32.27,NaN,NaN,NaN,NaN
